In [1]:
## SIMULATION PARAMETERS
verbosity         = 8        # Set debug print statement verbosity level (0 = Standard, -1 = Off)
use_mass_units    = True     # Toggle whether calculations / results are given in units of pi-axion mass (True) or eV (False)
use_natural_units = True     # Toggle whether calculations / results are given in c = h = G = 1 (True) or SI units (False)   || NOTE: full SI/phsyical unit support is still WIP!!
save_output_files = True     # Toggle whether or not the results from this notebook run are written to a data directory

## CONFIG PARAMETERS
config_name = 'piaxi_debug'   # Descriptive name for the given parameter case. Output files will be saved in a directory with this name.
seed = 269934808235809364794393739371438879116                      # rng_seed, integer value (None for random)
#seed = 178086272711189452008577645922249303295
num_cores = 100               # Number of parallel threads available
density = "1e20"              # Local DM energy density (in [Gev/cm^3])
method = "RK45"               # numerical integration method, to be passed to scipy.solve_ivp()

## Scenario to run
# NOTE: () means alias of previous listed item, [] indicates WIP/TODO scenarios.
scenario = "SIMPLE"           # one of: SINGLE, QCD, (AXION), SIMPLE, FULL, SU3, SU6, (SAMPLED), [REALS, NEUTRALS, COMPLEXES, CHARGED]

In [2]:
#%cd ..

In [3]:
#%load piaxiverse.py

In [4]:
#%conda init
#%conda activate piaxiverse

In [5]:
%%bash -s {density} {config_name} {seed} {num_cores} {verbosity} {use_mass_units} {method} {scenario} {save_output_files}

#module load anaconda
module load texlive
source /gpfs/runtime/opt/anaconda/latest/etc/profile.d/conda.sh
#conda activate piaxiverse

PIAXI_VERBOSITY=$5

if [[ "$PIAXI_VERBOSITY" -gt "6" ]]
then
    conda info
fi

if [[ "$6" = "False" ]]
then
    PIAXI_UNITS="--no-use_mass_units --no-use_natural_units"
else
    PIAXI_UNITS="--use_mass_units --use_natural_units"
fi

if [[ "$3" = "None" ]]
then
    PIAXI_SEED=""
else
    PIAXI_SEED="--seed $3"
fi

INPUT_ARG1="${8:-"0"}"
PIAXI_JOB_SUFFIX=""
PIAXI_FIT=""
if [[ "$INPUT_ARG1" = "0" ]]
then
    PIAXI_DQMC="1 1 1 0 0 0"
elif [[ "$INPUT_ARG1" = "SINGLE" ]]
then
    PIAXI_DQMC="0.5 0.5 0 0 0 0"
    PIAXI_JOB_SUFFIX="_single"
elif [[ "$INPUT_ARG1" = "QCD" ]] || [[ "$INPUT_ARG1" = "AXION" ]]
then
    PIAXI_DQMC="0.5 0.5 0 0 0 0"
    PIAXI_JOB_SUFFIX="_qcd"
    PIAXI_FIT="--fit_QCD"
elif [[ "$INPUT_ARG1" = "SIMPLE" ]]
then
    PIAXI_DQMC="1 1 1 0 0 0"
    PIAXI_JOB_SUFFIX="_simple"
elif [[ "$INPUT_ARG1" = "FULL" ]]
then
    PIAXI_DQMC="1 1 1 1 1 1"
    PIAXI_JOB_SUFFIX="_full"
elif [[ "$INPUT_ARG1" = "SU3" ]]
then
    PIAXI_DQMC="x x x 0 0 0"
    PIAXI_JOB_SUFFIX="_SU3"
elif [[ "$INPUT_ARG1" = "SAMPLED" ]] || [[ "$INPUT_ARG1" = "SU6" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_SU6"
fi

PIAXI_SYS_NAME="$2${PIAXI_JOB_SUFFIX}"
PIAXI_N_CORES=$SLURM_JOB_CPUS_PER_NODE
PIAXI_N_NODES=$SLURM_JOB_NUM_NODES
PIAXI_COREMEM=$SLURM_MEM_PER_NODE
PIAXI_JOB_QOS=$SLURM_JOB_QOS

if [[ "True" = $9 ]]
then
    PIAXI_SAVE="--save_output_files --make_plots"
else
    PIAXI_SAVE="--no-save_output_files --no-make_plots"
fi

PIAXI_N_TIMES=300
PIAXI_T_MAX=30
PIAXI_N_KMODE=200
PIAXI_K_RES=0.1

# || For QCD axion case:
# || Expect resonance for cases where [sqrt(2*Rho)/m_a] >= [F_pi]
# ||   --> let m_a = 1e-6 eV
# ||   --> let g_a = 1e-10 GeV^-1
# || Then: expect critical threshold between rho ~ 1e17 GeV and 1e18 GeV

# Density [GeV] || For QCD axion case ~ (amp_a)^2*m_a / 2
PIAXI_DENSITY=$1
# F_pi [GeV] || For QCD axion case ~ 2/g_a
PIAXI_F="1e10"
# m_I [eV] || For QCD axion case ~ (m_a)^2 / F_pi
PIAXI_MASS="1e-80"

# Pi-Axiverse coupling constants: Lambda_3 and Lambda_4 [GeV]
PIAXI_L3="1e0"
PIAXI_L4="1e10"

PIAXI_EPS="1e0"
PIAXI_METHOD=$7

python piaxiverse.py $PIAXI_SEED $PIAXI_UNITS --int_method $PIAXI_METHOD --no-skip_existing --num_cores $PIAXI_N_CORES --job_qos $PIAXI_JOB_QOS --mem_per_core $PIAXI_COREMEM --num_samples 1 --verbosity $PIAXI_VERBOSITY --t $PIAXI_T_MAX --tN $PIAXI_N_TIMES --kN $PIAXI_N_KMODE --k_res $PIAXI_K_RES --m_scale $PIAXI_MASS --config_name $PIAXI_SYS_NAME --eps $PIAXI_EPS --L3 $PIAXI_L3 --L4 $PIAXI_L4 --rho $PIAXI_DENSITY --dqm_c $PIAXI_DQMC --F $PIAXI_F $PIAXI_FIT $PIAXI_SAVE



     active environment : piaxiverse
    active env location : /users/sloane1/.conda/envs/piaxiverse
            shell level : 1
       user config file : /users/sloane1/.condarc
 populated config files : /gpfs/runtime/opt/anaconda/2022.05/.condarc
          conda version : 4.12.0
    conda-build version : 3.21.8
         python version : 3.9.12.final.0
       virtual packages : __linux=5.14.0=0
                          __glibc=2.34=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /gpfs/runtime/opt/anaconda/2022.05  (read only)
      conda av data dir : /gpfs/runtime/opt/anaconda/2022.05/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : 

100%|██████████| 1991/1991 [00:27<00:00, 71.83it/s] 


0:00:45.804311 elapsed on 100 cores
t_in: (300,) float64
n_in: (300,) float64
n_fit: (300,)
n_max: 207057218675086.53
n_min: 252088.2274153448
n_ini: 252088.2274153448
n_fin: 45850100023247.42
fit = a: 0.04   b: 0.80   c: 0.04
pcov = 
 [[ 0.00110052  0.         -0.00103449]
 [ 0.          0.          0.        ]
 [-0.00103449  0.          0.00110052]]
log-mean-squared error: 16.41
ratio_m:  821368061.4840279
ratio_f:  181881163.1679413
t_res:  24.88294314381271
t_max:  24.983277591973245
res_class:  resonance
max (peak) k mode: 1.0
max (mean) k mode: 1.0
fit = a: 0.04   b: 0.80   c: 0.04
pcov = 
 [[ 0.00110052  0.         -0.00103449]
 [ 0.          0.          0.        ]
 [-0.00103449  0.          0.00110052]]
log-mean-squared error: 16.41
tot_class: resonance
ratio_f: 181881163.1679413
ratio_m: 821368061.4840279
t_res: 24.88294314381271
t_max: 24.983277591973245
n_res: 33802235181.132652
n_max: 207057218675086.53
res  | t: 24.88    n = 3.38e+10
max  | t: 24.98    n = 2.07e+14
res co